In [1]:
# Import Necessary Packages

import numpy as np
import pandas as pd
import cv2
import imutils
import os
import shutil
import tensorflow
import keras
import matplotlib.pyplot as plt
import seaborn as sn
%matplotlib inline
import warnings
warnings.simplefilter('ignore')

# COMOUFLAGE DETECTIONS

In [2]:
Orig_Input_Dataset=r"F:\8k_normal_vs_camouflage_clothes_images"

In [3]:
Base_Path=r"F:\Camos And Not Camos Dataset"

In [4]:
Train_Path=os.path.sep.join([Base_Path,'training'])
Val_Path=os.path.sep.join([Base_Path,'validation'])
Test_Path=os.path.sep.join([Base_Path,'testing'])

In [5]:
Train_Split=0.75
Val_Split=0.1
classes=['camouflage_clothes','normal_clothes']

In [6]:
import random
from imutils import paths
imagePaths=list(paths.list_images(Orig_Input_Dataset))
random.seed(42)
random.shuffle(imagePaths)

In [7]:
i=int(len(imagePaths)*Train_Split)
trainPaths=imagePaths[:i]
testPaths=imagePaths[i:]

In [8]:
i=int(len(trainPaths)*Val_Split)
valPaths=trainPaths[:i]
trainPaths=trainPaths[i:]

In [9]:
datasets=[('training',trainPaths,Train_Path),
          ('validation',valPaths,Val_Path),
          ('testing',testPaths,Test_Path)]

In [10]:
for (dType,imagePaths,baseOutput) in datasets:
    print("[INFO] building '{}' split".format(dType))
    
    if not os.path.exists(baseOutput):
        print("[INFO] 'creating {}' directory".format(baseOutput))
        os.makedirs(baseOutput)
        
    for inputPath in imagePaths:
        filename=inputPath.split(os.path.sep)[-1]
        label=inputPath.split(os.path.sep)[-2]
        
        labelPath=os.path.sep.join([baseOutput,label])
        
        if not os.path.exists(labelPath):
            print("[INFO] 'creating {}' directory".format(labelPath))
            os.makedirs(labelPath)
            
        p=os.path.sep.join([labelPath,filename])
        shutil.copy2(inputPath,p)
        

[INFO] building 'training' split
[INFO] building 'validation' split
[INFO] building 'testing' split


In [11]:
totalTrain=len(list(paths.list_images(Train_Path)))
totalVal=len(list(paths.list_images(Val_Path)))
totalTest=len(list(paths.list_images(Test_Path)))

In [12]:
totalTrain

10731

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
trainAug=ImageDataGenerator(rotation_range=25,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")

In [15]:
valAug=ImageDataGenerator()

In [16]:
mean=np.array([123.68,116.779,103.939],dtype='float32')
trainAug.mean=mean
valAug.mean=mean

In [17]:
trainGen=trainAug.flow_from_directory(Train_Path,
                                      class_mode='categorical',
                                      target_size=(224,224),
                                      color_mode='rgb',
                                      shuffle=True,
                                      batch_size=32)

valGen=valAug.flow_from_directory(Val_Path,
                                class_mode='categorical',
                                  target_size=(224,224),
                                  color_mode='rgb',
                                  shuffle=True,
                                  batch_size=32)

testGen=valAug.flow_from_directory(Test_Path,
                                   class_mode='categorical',
                                   target_size=(224,224),
                                   color_mode='rgb',
                                   shuffle=True,
                                   batch_size=32)

Found 10731 images belonging to 2 classes.
Found 1192 images belonging to 2 classes.
Found 3975 images belonging to 2 classes.


In [18]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input,AveragePooling2D,BatchNormalization,Dense,Dropout,Flatten

In [19]:
basemodel=ResNet50(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))

In [20]:
basemodel.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [21]:
for layer in basemodel.layers:
    layer.trainable=False

In [22]:
headmodel=basemodel.output
headmodel=AveragePooling2D(pool_size=(7,7))(headmodel)
headmodel=Flatten(name='flatten')(headmodel)
headmodel=Dense(256,activation='relu')(headmodel)
headmodel=Dropout(0.5)(headmodel)
headmodel=Dense(len(classes),activation='sigmoid')(headmodel)

model=Model(inputs=basemodel.input,outputs=headmodel)

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [24]:
from tensorflow.keras.optimizers import Adam
opt=Adam(learning_rate=1e-4,decay=1e-4/10)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [26]:
H=model.fit_generator(trainGen,
                      steps_per_epoch=totalTrain//32,
                      validation_data=valGen,
                      validation_steps=totalVal//32,
                      epochs=10)

Epoch 1/10
335/335 [==============================] - ETA: 0s - loss: 0.1576 - accuracy: 0.9489WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x000001968033B708> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
335/335 [==============================] - 1085s 3s/step - loss: 0.1576 - accuracy: 0.9489 - val_loss: 0.0703 - val_accuracy: 0.97

In [27]:
print("Evaluating")
testGen.reset()
predIxs=model.predict_generator(testGen,steps=(totalTest//32)+1)

Evaluating
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [28]:
predIxs=np.argmax(predIxs,axis=1)

In [29]:
from sklearn.metrics import classification_report
print(classification_report(testGen.classes,predIxs,target_names=testGen.class_indices.keys()))

                    precision    recall  f1-score   support

camouflage_clothes       0.49      0.48      0.48      1968
    normal_clothes       0.50      0.52      0.51      2007

          accuracy                           0.50      3975
         macro avg       0.50      0.50      0.50      3975
      weighted avg       0.50      0.50      0.50      3975



In [31]:
model.save('camo_detector.h5')

In [2]:
from tensorflow.keras.models import load_model

# AGE DETECTION AND FACE DETECTION

In [3]:
protoPath=r"F:\Face Recognition\CAFFEMODEL\deploy.prototxt.txt"
configPath=r"F:\Face Recognition\CAFFEMODEL\res10_300x300_ssd_iter_140000.caffemodel\res10_300x300_ssd_iter_140000.caffemodel"

agedeploy_path=r"F:\Age Detection\gad\age_deploy.prototxt"
ageWeight=r"F:\Age Detection\gad\age_net.caffemodel"

In [4]:
args={'confidence':0.5}

In [5]:
from numpy import argmax
def detect_and_predict_age(image,faceNet,ageNet,min_conf=0.5):
    age_buckets=["(0-2)","(4-6)","(8-12)","(15-20)","(25-32)","(38-43)","(48-53)","(60-100)"]
    results=[]
    (h,w)=image.shape[:2]
    
    blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))
    faceNet.setInput(blob)
    detections=faceNet.forward()
    
    for i in range(0,detections.shape[2]):
        classId=int(detections[0,0,i,1])
        confidence=detections[0,0,i,2]
        
        if confidence>min_conf:
            box=detections[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype('int')
            
            face=image[startY:endY,startX:endX]
            if face.shape[0]<20 or face.shape[1]<20:
                continue
            faceblob=cv2.dnn.blobFromImage(face,1.0,(227,227),(78.4263377603, 87.7689143744, 114.895847746),swapRB=False)
            ageNet.setInput(faceblob)
            preds=ageNet.forward()
            i=preds[0].argmax()
            age=age_buckets[i]
            age_confidence=preds[0][i]
            
            d={'locs':(startX,startY,endX,endY),
               'age':(age,age_confidence)}
            
            results.append(d)
            
    return results
            

In [6]:
def detect_camo(image,camoNet):
    CLASS_LABELS=['camouflage_clothes','normal_clothes']
    Mean=np.array([123.68,116.779,103.939],dtype='float32')
    
    image=cv2.resize(image,(224,224))
    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image=np.expand_dims(image,axis=0).astype('float32')
    
    image-=Mean
    
    preds=camoNet.predict(image)[0]
    i=np.argmax(preds)
    
    return (CLASS_LABELS[i],preds[i])

# BLUR FACES

In [7]:
def anonymize_face_pixelate(image,blocks=3):
    (h,w)=image.shape[:2]
    xSteps=np.linspace(0,w,blocks+1,dtype='int')
    ySteps=np.linspace(0,h,blocks+1,dtype='int')
    
    for i in range(1,len(ySteps)):
        for j in range(1,len(xSteps)):
            startX=xSteps[j-1]
            startY=ySteps[i-1]
            endX=xSteps[j]
            endY=ySteps[i]
            
            roi=image[startY:endY,startX:endX]
            (B,G,R)=[int(x) for x in cv2.mean(roi)[:3]]
            cv2.rectangle(image,(startX,startY),(endX,endY),(B,G,R),-1)
    return image

In [ ]:
!pip install progressbar

# Implementing potential child soldier detector using OpenCV and Keras/TensorFlow

In [8]:
import progressbar
import cv2
import os

In [9]:
args={'dataset':r"F:\OCR\Child Soldier Images",
      'output':r"F:\Camouflage csv"}

In [10]:
Files={}

for k in ('ages','camo'):
    p=os.path.sep.join([args['output'],"{}.csv".format(k)])
    f=open(p,'w')
    Files[k]=f

In [11]:
faceNet=cv2.dnn.readNet(protoPath,configPath)
ageNet=cv2.dnn.readNet(agedeploy_path,ageWeight)

camoNet=load_model('camo_detector.h5')

In [12]:
from imutils import paths
imagePaths=sorted(list(paths.list_images(args['dataset'])))
print("[INFO] processing {} images".format(len(imagePaths)))

widgets=["Processing Images: ",progressbar.Percentage(),' ',progressbar.Bar(),' ',progressbar.ETA()]
pbar=progressbar.ProgressBar(maxval=len(imagePaths),widgets=widgets).start()

[INFO] processing 18 images


In [13]:
for (i,imagePath) in enumerate(imagePaths):
    image=cv2.imread(imagePath)
    
    if image is None:
        continue
        
    ageResults=detect_and_predict_age(image,faceNet,ageNet)
    
    camoResults=detect_camo(image,camoNet)
    
    for r in ageResults:
        row=[imagePath,*r['locs'],r['age'][0],r['age'][1]]
        row=','.join([str(x) for x in row])
        
        Files['ages'].write("{}\n".format(row))
        Files['ages'].flush()
        
    if camoResults[0]=='camouflage_clothes':
        row=[imagePath,camoResults[1]]
        row=','.join([str(x) for x in row])
        
        Files['camo'].write("{}\n".format(row))
        Files['camo'].flush()
        
    pbar.update(i)
    
pbar.finish()
print("[INFO] cleaning up.....")

for f in Files.values():
    f.close()

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 0s 246ms/step


1/1 [==============================] - 0s 251ms/step


1/1 [==============================] - 0s 207ms/step


1/1 [==============================] - 0s 188ms/step


...

1/1 [==============================] - 0s 189ms/step


1/1 [==============================] - 0s 187ms/step


1/1 [==============================] - 0s 160ms/step


1/1 [==============================] - 0s 168ms/step


[INFO] cleaning up.....


In [14]:
args={'ages':r"F:\Camouflage csv\ages.csv",
      'camo':r"F:\Camouflage csv\camo.csv"}

In [15]:
agerows=open(args['ages']).read().strip().split('\n')
camorows=open(args['camo']).read().strip().split('\n')

In [16]:
ages={}
camo={}

for row in agerows:
    row=row.split(",")
    imagePath=row[0]
    bbox=[int(x) for x in row[1:5]]
    age=row[5]
    ageProb=float(row[6])
    
    t=(bbox,age,ageProb)
    
    l=ages.get(imagePath,[])
    l.append(t)
    ages[imagePath]=l
    
for row in camorows:
    row=row.split(',')
    imagePath=row[0]
    camoProb=float(row[1])
    
    camo[imagePath]=camoProb
    
inter=sorted(set(ages.keys()).intersection(camo.keys()))

for imagePath in inter:
    image=cv2.imread(imagePath)
    (h,w)=image.shape[:2]
    
    if w>h:
        image=imutils.resize(image,width=600)
    else:
        image=imutils.resize(image,width=600)
        
    ratio=image.shape[1]/float(w)
    
    
    for (bbox,age,ageProb) in ages[imagePath]:
        bbox=[int(x) for x in np.array(bbox)*ratio]
        (startX,startY,endX,endY)=bbox
    
        face=image[startY:endY,startX:endX]
        face=anonymize_face_pixelate(face,blocks=5)
        image[startY:endY,startX:endX]=face
    
        color=(0,255,0)
    
        if age in ["(0-2)", "(4-6)", "(8-12)", "(15-20)"]:
            color=(0,0,255)
        
        text="{}:{:.2f}%".format(age,ageProb*100)
        y=startY-10 if startY-10>10 else startY+10
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        cv2.putText(image,text,(startX,y),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        
    label="camo:{:.2f}%".format(camo[imagePath]*100)
    cv2.rectangle(image, (0, 0), (300, 40), (0, 0, 0), -1)
    cv2.putText(image, label, (10, 25), cv2.FONT_HERSHEY_SIMPLEX,0.8, (255, 255, 255), 2)
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    
    
    